# NUS DSA Module Planning Assistant — API Tooling

This notebook wires up the NUS Data Science & Analytics planning chatbot with the API tooling described in the project specification.

It focuses on the LangGraph agent shell and the NUSMods REST API tools. Context retrieval tools will be integrated later.

Use the annotated sections below to understand how each component contributes to the end-to-end conversation loop—starting from HTTP requests, through LangChain tool wrappers, and finally into the LangGraph state machine that powers the chat experience.


In [1]:
# Core language features and typing helpers
from __future__ import annotations

# Logging keeps the API interactions transparent when debugging
import logging
from typing import Any, Dict, Iterable, List, Optional

# Third-party dependencies for HTTP access and the LangChain/LangGraph stack
import requests
from langchain_core.tools import tool
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, ToolMessage
from langchain_ollama.chat_models import ChatOllama
from langgraph.graph import START, MessagesState, StateGraph
from langgraph.prebuilt import ToolNode, tools_condition


In [2]:
# Configure a module-specific logger so network activity is easy to inspect.
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("nusmods")


class NusModsClient:
    """Thin wrapper around the NUSMods v2 API with simple response caching."""

    def __init__(self, default_acad_year: str = "2025-2026"):
        # Default AY so every tool can omit it unless the user specifies otherwise.
        self.default_acad_year = default_acad_year
        # Single `Session` reuses TCP connections for faster repeated calls.
        self._session = requests.Session()
        # In-memory caches prevent duplicate requests when the same module is referenced.
        self._module_cache: Dict[str, Dict[str, Any]] = {}
        self._module_list_cache: Dict[str, List[Dict[str, Any]]] = {}

    def _year_base(self, acad_year: Optional[str]) -> str:
        year = acad_year or self.default_acad_year
        return f"https://api.nusmods.com/v2/{year}"

    def _normalise_code(self, module_code: str) -> str:
        code = (module_code or "").strip().upper()
        if not code:
            raise ValueError("module_code is required")
        return code

    def module(self, module_code: str, acad_year: Optional[str] = None) -> Dict[str, Any]:
        # Fetch the canonical module JSON payload (and memoise it).
        code = self._normalise_code(module_code)
        year = acad_year or self.default_acad_year
        cache_key = f"{year}:{code}"
        if cache_key not in self._module_cache:
            url = f"{self._year_base(year)}/modules/{code}.json"
            response = self._session.get(url)
            response.raise_for_status()
            self._module_cache[cache_key] = response.json()
        return self._module_cache[cache_key]

    def module_list(self, acad_year: Optional[str] = None) -> List[Dict[str, Any]]:
        # Entire catalogue dump (cached) that powers keyword search.
        year = acad_year or self.default_acad_year
        if year not in self._module_list_cache:
            url = f"{self._year_base(year)}/moduleList.json"
            response = self._session.get(url)
            response.raise_for_status()
            self._module_list_cache[year] = response.json()
        return self._module_list_cache[year]

    def search_modules(
        self,
        query: str,
        acad_year: Optional[str] = None,
        level: Optional[int] = None,
        limit: int = 10,
    ) -> List[Dict[str, Any]]:
        # Basic keyword search across module codes and titles with optional level filtering.
        query_lower = (query or "").strip().lower()
        if not query_lower:
            raise ValueError("query must be a non-empty string")

        matches: List[Dict[str, Any]] = []
        for mod in self.module_list(acad_year):
            if level is not None:
                code = mod.get("moduleCode", "")
                if len(code) >= 3 and code[2].isdigit():
                    if int(code[2]) != int(level):
                        continue
                else:
                    continue
            if query_lower in mod.get("moduleCode", "").lower() or query_lower in mod.get("title", "").lower():
                matches.append(mod)
            if len(matches) >= limit:
                break
        return matches

    def module_timetable(
        self,
        module_code: str,
        acad_year: Optional[str] = None,
        semester: Optional[int] = None,
    ) -> List[Dict[str, Any]]:
        # Pull the semester-by-semester timetable blocks for a module.
        data = self.module(module_code, acad_year)
        semester_data = data.get("semesterData", [])
        if semester is None:
            return semester_data
        return [sem for sem in semester_data if sem.get("semester") == semester]


# Shared client instance reused across all LangChain tools.
client = NusModsClient()


## LangChain tool wrappers

Each decorated function below exposes a focused slice of the NUSMods API.
They are intentionally lightweight so the language model can call them with the
exact parameters needed to answer a student query.


In [3]:
@tool
def nusmods_module_overview(module_code: str, acad_year: Optional[str] = None) -> Dict[str, Any]:
    """Retrieve the canonical module payload for course planning questions.

    Use this tool whenever you need authoritative facts about a module, such as its
    title, MC value, summary description, faculty ownership, or the semesters when
    it typically runs. The response is trimmed for conversational use so the model
    does not need to wade through the full API schema.

    Inputs:
    - module_code (str): NUS module code like "CS3244". Required.
    - acad_year (str, optional): Academic year in "YYYY-YYYY" format. Defaults
      to the client's configured year when omitted.

    Output (dict):
    - moduleCode: Normalised module code.
    - title: Official module title.
    - description: Module synopsis text.
    - moduleCredit: MC value as a string.
    - faculty / department: Owning faculty metadata.
    - prerequisite / preclusion / fulfillRequirements: Relationship metadata that
      helps answer follow-up eligibility questions.
    Use `nusmods_module_timetable` when you need semester-specific lesson timings.
    """
    # The client fetches and trims the verbose payload to the pieces planners need.
    data = client.module(module_code, acad_year)
    return {
        "moduleCode": data.get("moduleCode"),
        "title": data.get("title"),
        "description": data.get("description"),
        "moduleCredit": data.get("moduleCredit"),
        "faculty": data.get("faculty"),
        "department": data.get("department"),
        "prerequisite": data.get("prerequisite"),
        "preclusion": data.get("preclusion"),
        "fulfillRequirements": data.get("fulfillRequirements"),
    }


@tool
def nusmods_module_prerequisites(module_code: str, acad_year: Optional[str] = None) -> Dict[str, Any]:
    """Surface prerequisite, preclusion, and fulfilment data for a module.

    Use this tool when the student is checking eligibility, dependency chains,
    or which later modules list the target as a prerequisite. The schema narrows
    the NUSMods payload down to the relationship fields that matter for advising.

    Inputs:
    - module_code (str): NUS module code being evaluated. Required.
    - acad_year (str, optional): Academic year in "YYYY-YYYY" format. Defaults to
      the client's configured year when omitted.

    Output (dict):
    - moduleCode: Normalised module code.
    - title: Module title for context in the response.
    - prerequisite: Human-readable prerequisite description.
    - prerequisiteTree: Structured prerequisite tree usable for reasoning.
    - fulfillRequirements: List of modules that accept this module as fulfilment.
    - preclusion / corequisite: Additional relationship metadata to mention if relevant.
    """
    # Keep the focus on dependency-related keys so the agent can reason about eligibility.
    data = client.module(module_code, acad_year)
    return {
        "moduleCode": data.get("moduleCode"),
        "title": data.get("title"),
        "prerequisite": data.get("prerequisite"),
        "prerequisiteTree": data.get("prerequisiteTree"),
        "fulfillRequirements": data.get("fulfillRequirements"),
        "preclusion": data.get("preclusion"),
        "corequisite": data.get("corequisite"),
    }


@tool
def nusmods_module_timetable(
    module_code: str,
    acad_year: Optional[str] = None,
    semester: Optional[int] = None,
    limit_lessons: Optional[int] = 20,
) -> Dict[str, Any]:
    """Summarise the module timetable across semesters and lesson groupings.

    Call this tool when the question involves class availability, lesson timings,
    or exam dates for a specific semester. It can optionally filter to one semester
    and clamps the number of raw lesson rows so the response stays manageable.

    Inputs:
    - module_code (str): NUS module code to inspect. Required.
    - acad_year (str, optional): Academic year in "YYYY-YYYY" format. Defaults to
      the client's configured year when omitted.
    - semester (int, optional): Semester number (1 or 2). When omitted, returns all
      semesters available in the academic year.
    - limit_lessons (int, optional): Maximum timetable rows to include per semester.
      Use a smaller value if the schedule is extremely long.

    Output (dict):
    - moduleCode: Normalised module code.
    - acadYear: Academic year used for the lookup.
    - semesterData: List with one entry per semester that contains:
        * semester: Semester number.
        * lessons: Timetable entries with class number, activity type, day, and time.
    """
    # Normalise the nested structure into a concise dictionary for the LLM.
    semester_data = client.module_timetable(module_code, acad_year, semester)
    shaped: List[Dict[str, Any]] = []
    for sem in semester_data:
        lessons = sem.get("timetable", [])
        if limit_lessons is not None:
            lessons = lessons[:limit_lessons]
        shaped.append({
            "semester": sem.get("semester"),
            "lessons": lessons,
        })
    return {
        "moduleCode": client._normalise_code(module_code),
        "acadYear": acad_year or client.default_acad_year,
        "semesterData": shaped,
    }


@tool
def nusmods_module_search(
    query: str,
    acad_year: Optional[str] = None,
    level: Optional[int] = None,
    limit: int = 10,
) -> Dict[str, Any]:
    """Locate modules by keyword, optionally filtered by level, for discovery tasks.

    Trigger this tool when the student is exploring potential electives, looking
    for modules that match a theme, or asking for options at a specific level.
    The search is performed client-side against the cached module list so repeat
    queries remain responsive within a notebook session.

    Inputs:
    - query (str): Keyword to match against module codes and titles. Required.
    - acad_year (str, optional): Academic year in "YYYY-YYYY" format. Defaults to
      the client's configured year when omitted.
    - level (int, optional): Restrict results to a numeric module level (e.g. 1, 2, 3).
    - limit (int, optional): Maximum number of results to return. Defaults to 10.

    Output (dict):
    - query: Echo of the original search term.
    - acadYear: Academic year used for the search.
    - count: Number of modules returned.
    - results: List of module summaries, each containing moduleCode, title, and MCs.
    """
    # Search reuses the cached module list for responsive autocomplete-style experiences.
    matches = client.search_modules(query, acad_year, level=level, limit=limit)
    return {
        "query": query,
        "acadYear": acad_year or client.default_acad_year,
        "count": len(matches),
        "results": [
            {
                "moduleCode": mod.get("moduleCode"),
                "title": mod.get("title"),
                "moduleCredit": mod.get("moduleCredit"),
            }
            for mod in matches
        ],
    }


API_TOOLS = [
    nusmods_module_overview,
    nusmods_module_prerequisites,
    nusmods_module_timetable,
    nusmods_module_search,
]


## Retrieve Information

In [77]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.messages import HumanMessage, AIMessage

embedding = OllamaEmbeddings(model="mxbai-embed-large") 

vectorstore = FAISS.load_local("curriculum_info_vectors", embedding, allow_dangerous_deserialization=True)  # pass the same embedding instance
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4})

def retrieve_node(state):
    print("🔹 Running retrieve_node")
    query = state["messages"][-1].content  # last user message
    docs = retriever.invoke(query)
    #state["messages"].append(AIMessage(content=f"Retrieved {len(docs)} document(s)."))
    print(f"Retrieved {len(docs)} document(s).")
    return {"retrieved_docs": docs}


## LLM configuration

The assistant is powered by an Ollama-served Qwen 3 14B model with reasoning enabled.
We bind the previously defined tools so the model can call them via function-calling.


In [56]:
# Instantiate the local Ollama model and attach the tool schema.
llm = ChatOllama(
    model="qwen3:14b",
    temperature=0.2,
    num_predict=-1,
    reasoning=True,
    validate_model_on_init=True,
)

# Tool binding enables structured tool-calling responses.
llm_with_tools = llm.bind_tools(API_TOOLS)

# System prompt keeps the agent grounded in planning responsibilities.
system_prompt = SystemMessage(
    content=(
        "You are an academic planning assistant for the NUS Data Science & Analytics major. "
        "Always review the full chat history so follow-up questions stay consistent. "
        "Use a private chain-of-thought to break complex requests into sub-questions, plan the tool-call sequence, and call multiple tools when needed before answering. "
        "If a student's question is ambiguous or missing critical details, ask for clarification before committing to a tool plan. "
        "If a student's question does not specify an Academic year, assume the current: 2025-2026."
        "Ground every module fact in the provided NUSMods API tools and cross-check conflicting data. "
        "If a question falls outside academic planning, politely steer the student back to relevant topics. "
        "If a module cannot be located, apologise and suggest verifying the code or academic year, "
        "and if the tools cannot answer, explain the limitation instead of guessing."
        "If external information have been retrieved, make use of those information."
    )
)

INFO:httpx:HTTP Request: GET http://127.0.0.1:11434/api/tags "HTTP/1.1 200 OK"


## LangGraph conversation loop

This section wires the language model and the toolset into a two-node graph:
the assistant decides whether to answer directly or call a tool, and the tool
node executes the request before control returns to the model.


In [78]:
# LangGraph node that calls the LLM with the accumulated chat history.

class ModState(MessagesState):
    messages: any = None
    retrieved_docs: any = None
    router_decision: any = None

def assistant(state: ModState):
    print(state.get("retrieved_docs"))
    context = ""
    if state.get("retrieved_docs"):
        context = "\n".join(doc.page_content for doc in state.get("retrieved_docs"))
        
    last_user_msg = state.get("messages")[-1].content
    augmented_input = f"Context:\n{context}\n\nUser: {last_user_msg}"

    reply = llm_with_tools.invoke([system_prompt] + [{"role": "user", "content": augmented_input}])
    return {"messages": [reply]}

def router_node(state: ModState):
    query = state.get("messages")[-1].content

    retrieved_texts = ""

    if state.get("retrieved_docs"):
        # Combine retrieved document summaries
        retrieved_texts = "\n\n".join(
            f"Doc {i+1}: {doc.page_content[:500]}" for i, doc in enumerate(state.get("retrieved_docs"))
        )

    router_prompt = f"""
    You are a routing agent helping a course-planning assistant.

    The user asked:
    "{query}"

    The assistant already has access to these retrieved documents:
    {retrieved_texts if retrieved_texts else "[No documents retrieved yet]"}

    Decide if **more retrieval is needed** to answer the question accurately.
    Respond with only one word:
    - "retrieve" → if additional or updated retrieval is necessary
    - "proceed" → if current context and retrieved_docs are sufficient
    """

    decision = llm.invoke([SystemMessage(content=router_prompt)]).content.strip().lower()

    if "retrieve" in decision:
        return {"router_decision": "retrieve_requirements"}
    else:
        return {"router_decision": "assistant"}
    


builder = StateGraph(ModState)

builder.add_node("router", router_node)
builder.add_node("assistant", assistant)
builder.add_node("retrieve_requirements", retrieve_node)
builder.add_node("tools", ToolNode(API_TOOLS))

# Start from router
builder.add_edge(START, "router")

# Router decides whether to retrieve or proceed
builder.add_conditional_edges(
    "router",
    lambda state: state.get("router_decision"),
    {
        "retrieve_requirements": "retrieve_requirements",
        "assistant": "assistant",
    },
)

# Normal flow after retrieval
builder.add_edge("retrieve_requirements", "assistant")
builder.add_conditional_edges("assistant", tools_condition)
builder.add_edge("tools", "assistant")

graph = builder.compile()

## Chat helpers for notebook exploration

The utilities below maintain a bounded conversation state, stream intermediate
tool traces, and expose helper functions so you can quickly test prompts.


In [7]:
# Keep a rolling chat history so the agent has short-term memory.
MAX_HISTORY = 5
chat_state: Dict[str, Any] = {"messages": []}


def _trim(messages: Iterable[Any]) -> List[Any]:
    """Limit the stored history to the most recent MAX_HISTORY turns."""
    max_messages = MAX_HISTORY * 2
    if max_messages <= 0:
        return []
    seq = list(messages)
    if len(seq) <= max_messages:
        return seq
    return seq[-max_messages:]


def _condense_history(messages: Iterable[Any]) -> List[Any]:
    """Condense prior turns to just the human prompt and final assistant reply."""
    condensed: List[Any] = []
    pending_human: Optional[HumanMessage] = None
    for message in messages:
        if isinstance(message, HumanMessage):
            pending_human = HumanMessage(content=getattr(message, "content", str(message)))
        elif isinstance(message, AIMessage):
            ai_message = AIMessage(content=getattr(message, "content", str(message)))
            if pending_human is not None:
                condensed.extend([pending_human, ai_message])
                pending_human = None
            else:
                condensed.append(ai_message)
    if pending_human is not None:
        condensed.append(pending_human)
    return _trim(condensed)


def _msg_type(message: Any) -> str:
    """Pretty-print helper for LangChain message objects."""
    return getattr(message, "type", message.__class__.__name__).upper()


def _msg_text(message: Any) -> str:
    """Extract message body, handling ToolMessage payload differences."""
    if isinstance(message, ToolMessage):
        return str(message.content)
    return getattr(message, "content", str(message))


def _msg_metadata(message: Any) -> Dict[str, Any]:
    """Collect any auxiliary metadata attached to LangChain messages."""
    metadata: Dict[str, Any] = {}
    additional = getattr(message, "additional_kwargs", None)
    if additional:
        metadata["additional_kwargs"] = additional
    response_meta = getattr(message, "response_metadata", None)
    if response_meta:
        metadata["response_metadata"] = response_meta
    tool_calls = getattr(message, "tool_calls", None)
    if tool_calls:
        metadata["tool_calls"] = tool_calls
    return metadata


def reset_chat() -> None:
    """Clear the global chat state to restart the conversation."""
    global chat_state
    chat_state = {"messages": []}
    print("Chat state reset.")


def ask(prompt: str, show_trace: bool = True, developer_view: bool = False) -> None:
    """Submit a user message, optionally printing the intermediate LangGraph trace and diagnostics."""
    global chat_state

    show_trace = show_trace or developer_view
    history = _trim(chat_state["messages"] + [HumanMessage(content=prompt)])

    if developer_view:
        print("=== Developer View: Model Input ===")
        for idx, msg in enumerate([system_prompt] + history, start=1):
            print(f"{idx:02d}. [{_msg_type(msg)}] {_msg_text(msg)}")
            metadata = _msg_metadata(msg)
            if metadata:
                print(f"    metadata: {metadata}")
        print("=" * 40)

    last_len = len(history)
    final_state = None

    if show_trace:
        print("=== Stream trace ===")

    for state in graph.stream({"messages": history}, stream_mode="values"):
        msgs = state["messages"]
        new_msgs = msgs[last_len:]
        if show_trace and new_msgs:
            for msg in new_msgs:
                print(f"[{_msg_type(msg)}] {_msg_text(msg)}")
                if developer_view:
                    metadata = _msg_metadata(msg)
                    if metadata:
                        print(f"    metadata: {metadata}")
                print("-" * 40)
        last_len = len(msgs)
        final_state = state
        
    if show_trace:
        print("=== END of stream trace ===")
        
    if final_state is not None:
        condensed = _condense_history(final_state["messages"])
        chat_state = {"messages": condensed}

    if developer_view:
        print("=== Developer View: Stored Chat State ===")
        for idx, msg in enumerate(chat_state["messages"], start=1):
            print(f"{idx:02d}. [{_msg_type(msg)}] {_msg_text(msg)}")
            metadata = _msg_metadata(msg)
            if metadata:
                print(f"    metadata: {metadata}")
        print("=" * 40)
        print("=== Developer View: Final Answer ===")

    for msg in reversed(chat_state["messages"]):
        if isinstance(msg, AIMessage):
            print(msg.content)
            break

## Example runs

Use these sample prompts to sanity-check the tool orchestration.


In [27]:
# Start from a clean slate (clear chat history).
reset_chat()
# Test problem-solving capabilities (ability to break down question into smaller parts).
ask("What is the timetable for DSA4213 in sem 1 like? What are its prerequisites?")

Chat state reset.
=== Stream trace ===
🔹 Running retrieve_node
Retrieved 4 document(s).
[AI] Retrieved 4 document(s).
----------------------------------------


AttributeError: 'dict' object has no attribute 'retrieved_docs'

In [31]:
# Test follow-up capabilities.
ask("What about for DSA3101?", developer_view=True)

=== Developer View: Model Input ===
01. [SYSTEM] You are an academic planning assistant for the NUS Data Science & Analytics major. Always review the full chat history so follow-up questions stay consistent. Use a private chain-of-thought to break complex requests into sub-questions, plan the tool-call sequence, and call multiple tools when needed before answering. If a student's question is ambiguous or missing critical details, ask for clarification before committing to a tool plan. If a student's question does not specify an Academic year, assume the current: 2025-2026.Ground every module fact in the provided NUSMods API tools and cross-check conflicting data. If a question falls outside academic planning, politely steer the student back to relevant topics. If a module cannot be located, apologise and suggest verifying the code or academic year, and if the tools cannot answer, explain the limitation instead of guessing.
02. [HUMAN] What is the timetable for DSA4213 in sem 1 like? Wh

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


[AI] 
    metadata: {'additional_kwargs': {'reasoning_content': "Okay, the user previously asked about DSA4213's timetable and prerequisites, and now they're asking about DSA3101. I need to figure out what they want. Since their previous question covered both timetable and prerequisites, they might be asking for the same information about DSA3101. \n\nFirst, I should check if DSA3101 exists. The user might have a typo, but DSA3101 is a valid module code. Let me confirm using the nusmods_module_overview function. \n\nI'll start by calling nusmods_module_overview for DSA3101 to get the module's details. Then, I'll need to check the prerequisites using nusmods_module_prerequisites. Also, the timetable can be retrieved with nusmods_module_timetable. \n\nWait, the user didn't specify the academic year, so I'll use the default (2025-2026). For the timetable, I should check if there's a specific semester mentioned. The user didn't mention a semester, so I'll get the timetable for all semester

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


[AI] The timetable for **DSA3101 (Data Science in Practice)** in **Semester 1, 2025-2026** is as follows:

- **Lecture**:  
  - **Monday 18:00–21:00** (LT34)  
  *(Weekly for 13 weeks.)*  
- **Tutorial**:  
  - **Monday 21:00–22:00** (LT34)  
  *(Weekly for 13 weeks.)*  

---

**Prerequisites** (for undergraduates):  
- **DSA2101** with a grade of at least **D**.  
- **ST2132** with a grade of at least **D**.  

This module fulfills requirements for several advanced modules (e.g., **DSA4261**, **DSA4288**, **DSA4510**, etc.).  

Let me know if you need further details! 😊
    metadata: {'additional_kwargs': {'reasoning_content': "Okay, the user asked about DSA3101's timetable and prerequisites. Let me start by checking the tools I have. The previous response used three functions: module_overview, module_prerequisites, and module_timetable. The user's question is similar to the previous one about DSA4213, so I should follow the same approach.\n\nFirst, I need to get the timetable for DSA

In [32]:
# Test response to irrelevant questions
ask("Whats the weather today?", developer_view=True)

=== Developer View: Model Input ===
01. [SYSTEM] You are an academic planning assistant for the NUS Data Science & Analytics major. Always review the full chat history so follow-up questions stay consistent. Use a private chain-of-thought to break complex requests into sub-questions, plan the tool-call sequence, and call multiple tools when needed before answering. If a student's question is ambiguous or missing critical details, ask for clarification before committing to a tool plan. If a student's question does not specify an Academic year, assume the current: 2025-2026.Ground every module fact in the provided NUSMods API tools and cross-check conflicting data. If a question falls outside academic planning, politely steer the student back to relevant topics. If a module cannot be located, apologise and suggest verifying the code or academic year, and if the tools cannot answer, explain the limitation instead of guessing.
02. [HUMAN] What is the timetable for DSA4213 in sem 1 like? Wh

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


[AI] I currently don't have access to real-time weather data or external services to check the weather. However, I'm happy to help with academic planning questions related to modules, timetables, prerequisites, or other NUS Data Science & Analytics topics! 😊
    metadata: {'additional_kwargs': {'reasoning_content': 'Okay, the user asked, "Whats the weather today?" Let me check the tools provided. The available functions are related to module information: module overview, prerequisites, timetable, and search. There\'s no tool here for weather data. Since the user\'s question is about the weather, which isn\'t covered by any of the provided functions, I need to inform them that I can\'t help with that. I should politely steer them back to academic planning topics. Let me make sure there\'s no misunderstanding. The tools are all NUSMods related, so weather is definitely outside the scope. I\'ll respond accordingly.\n'}, 'response_metadata': {'model': 'qwen3:14b', 'created_at': '2025-10-20

In [79]:
reset_chat()

ask("What are the modules in chs common core communities and engagement pillar?", developer_view=True)

Chat state reset.
=== Developer View: Model Input ===
01. [SYSTEM] You are an academic planning assistant for the NUS Data Science & Analytics major. Always review the full chat history so follow-up questions stay consistent. Use a private chain-of-thought to break complex requests into sub-questions, plan the tool-call sequence, and call multiple tools when needed before answering. If a student's question is ambiguous or missing critical details, ask for clarification before committing to a tool plan. If a student's question does not specify an Academic year, assume the current: 2025-2026.Ground every module fact in the provided NUSMods API tools and cross-check conflicting data. If a question falls outside academic planning, politely steer the student back to relevant topics. If a module cannot be located, apologise and suggest verifying the code or academic year, and if the tools cannot answer, explain the limitation instead of guessing.If external information have been retrieved, m

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


🔹 Running retrieve_node
Retrieved 4 document(s).
[Document(id='209328d2-3376-46be-b590-11f6c9db143f', metadata={'group': 'CHS Common Core Modules', 'pillar': 'Communities and Engagement', 'category': 'Service-Learning C&E courses'}, page_content='Group: CHS Common Core Modules\nPillar Name: Communities and Engagement\nCategory: Service-Learning C&E courses\nSemester: 2\nModules: GEN2050Y (Teach SG), GEN2060Y (Reconnect SeniorsSG), GEN2061Y (Support Healthy AgeingSG), GEN2062Y (Community Activities for Seniors with SG Cares), GEN2070Y (Community Link (Comlink) Befrienders)'), Document(id='a4e0ac37-5df3-40fc-ba6d-db4407729a35', metadata={'group': 'CHS Common Core Modules', 'pillar': 'Communities and Engagement', 'category': 'Service-Learning C&E courses'}, page_content='Group: CHS Common Core Modules\nPillar Name: Communities and Engagement\nCategory: Service-Learning C&E courses\nSemester: 1\nModules: GEN2050X (Teach SG), GEN2060X (Reconnect SeniorsSG), GEN2061X (Support Healthy AgeingS

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


=== END of stream trace ===
=== Developer View: Stored Chat State ===
01. [AI] The CHS Common Core **Communities and Engagement** pillar includes the following modules, organized by category and semester:

---

### **1. Service-Learning C&E Courses**
#### **Semester 1**
- **GEN2050X** - Teach SG  
- **GEN2060X** - Reconnect SeniorsSG  
- **GEN2061X** - Support Healthy AgeingSG  
- **GEN2062X** - Community Activities for Seniors with SG Cares  
- **GEN2070X** - Community Link (Comlink) Befrienders  

#### **Semester 2**
- **GEN2050Y** - Teach SG  
- **GEN2060Y** - Reconnect SeniorsSG  
- **GEN2061Y** - Support Healthy AgeingSG  
- **GEN2062Y** - Community Activities for Seniors with SG Cares  
- **GEN2070Y** - Community Link (Comlink) Befrienders  

---

### **2. Project-based Engagement-Learning C&E Courses**
#### **Gen-Coded: False** (Non-GEN modules)
- **BN4102** - Gerontechnology in Ageing  
- **BN4103** - Assistive Technology for Persons with Disability  
- **CDE2001** - Innovation

In [80]:
ask("What are the timetables for service-learning c&e courses in semester 2")

=== Stream trace ===


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


🔹 Running retrieve_node
Retrieved 4 document(s).
[Document(id='da77d815-d468-407f-b243-174ba536da4b', metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2024-06-12T11:07:25+00:00', 'source': 'dsa_degree_requirements.pdf', 'total_pages': 5, 'page': 4, 'page_label': '5'}, page_content='Note on CHS Common Curriculum courses: \n \n1) Students are strongly encouraged to complete all CHS Common Curriculum courses in their first \ntwo years except for the following 3 courses: \n• Communities and Engagement course – can be taken from Years 2 to 4* \n• Two Interdisciplinary courses – can be taken in Years 3 and 4 \n \n*Important note on workload: Semester vs. Year-long C&E courses \n• Some C&E courses, usually the field/project-work courses, are regular intense 4-Unit courses \nwith work completed within one semester. \n• Other C&E courses, especially the service-work courses, are spread out over two consecutive \nsemesters, or up to one year, that is, Semeste

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


[TOOL] {"moduleCode": "GEN2060Y", "acadYear": "2025-2026", "semesterData": []}
----------------------------------------
[TOOL] {"moduleCode": "GEN2061Y", "acadYear": "2025-2026", "semesterData": []}
----------------------------------------
[TOOL] {"moduleCode": "GEN2062Y", "acadYear": "2025-2026", "semesterData": []}
----------------------------------------
[TOOL] {"moduleCode": "GEN2070Y", "acadYear": "2025-2026", "semesterData": []}
----------------------------------------
[Document(id='da77d815-d468-407f-b243-174ba536da4b', metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2024-06-12T11:07:25+00:00', 'source': 'dsa_degree_requirements.pdf', 'total_pages': 5, 'page': 4, 'page_label': '5'}, page_content='Note on CHS Common Curriculum courses: \n \n1) Students are strongly encouraged to complete all CHS Common Curriculum courses in their first \ntwo years except for the following 3 courses: \n• Communities and Engagement course – can be taken from Ye

...

[TOOL] {"moduleCode": "GEN2070Y", "acadYear": "2025-2026", "semesterData": []}
----------------------------------------
[Document(id='da77d815-d468-407f-b243-174ba536da4b', metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2024-06-12T11:07:25+00:00', 'source': 'dsa_degree_requirements.pdf', 'total_pages': 5, 'page': 4, 'page_label': '5'}, page_content='Note on CHS Common Curriculum courses: \n \n1) Students are strongly encouraged to complete all CHS Common Curriculum courses in their first \ntwo years except for the following 3 courses: \n• Communities and Engagement course – can be taken from Years 2 to 4* \n• Two Interdisciplinary courses – can be taken in Years 3 and 4 \n \n*Important note on workload: Semester vs. Year-long C&E courses \n• Some C&E courses, usually the field/project-work courses, are regular intense 4-Unit courses \nwith work completed within one semester. \n• Other C&E courses, especially the service-work courses, are spread ou

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


[Document(id='da77d815-d468-407f-b243-174ba536da4b', metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2024-06-12T11:07:25+00:00', 'source': 'dsa_degree_requirements.pdf', 'total_pages': 5, 'page': 4, 'page_label': '5'}, page_content='Note on CHS Common Curriculum courses: \n \n1) Students are strongly encouraged to complete all CHS Common Curriculum courses in their first \ntwo years except for the following 3 courses: \n• Communities and Engagement course – can be taken from Years 2 to 4* \n• Two Interdisciplinary courses – can be taken in Years 3 and 4 \n \n*Important note on workload: Semester vs. Year-long C&E courses \n• Some C&E courses, usually the field/project-work courses, are regular intense 4-Unit courses \nwith work completed within one semester. \n• Other C&E courses, especially the service-work courses, are spread out over two consecutive \nsemesters, or up to one year, that is, Semester 1 through Semester 2 to Special Term 2; or \nSe

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


=== END of stream trace ===
The timetable for **GEN2070Y (Community Link (Comlink) Befrienders)** in **Semester 1, 2025-2026** is as follows:

### **Lessons (Semester 1):**
- **Class 01**: Monday 14:00–16:00, ERC-SR10  
- **Class 02**: Tuesday 16:00–18:00, TP-SR9  
- **Class 03**: Wednesday 10:00–12:00, ERC-SR10  
- **Class 04**: Thursday 08:00–10:00, ERC-SR9CAM  
- **Class 05**: Thursday 10:00–12:00, ERC-SR9CAM  
- **Class 06**: Friday 12:00–14:00, ERC-SR9CAM  

All classes occur in **Week 11** of the semester.  

---

### **Important Notes:**
1. **Year-Long Course**: This is a **year-long C&E course** (spanning Semester 1, Semester 2, and Special Term 2). Grades are awarded at the end of **Special Term 2**, delaying degree conferral to **end-August** and postponing the Commencement ceremony to the following year.  
2. **Workload**: As a service-work C&E course, it involves sustained engagement over multiple semesters.  
3. **Planning**: Students are advised to include this course ear

In [72]:
ask("GEN2050X")

=== Stream trace ===


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


🔹 Running retrieve_node
Retrieved 4 document(s).
[Document(id='0d46abcd-7bb5-4549-8365-3d11c54665c9', metadata={'group': 'CHS Integrated Modules', 'pillar': 'Scientific Inquiry II'}, page_content='Group: CHS Integrated Modules\nPillar: Scientific Inquiry II\nModules: HSI20XX'), Document(id='7be994ef-3bd0-4300-92fd-8fcf5449e62c', metadata={'group': 'CHS Common Core Modules', 'pillar': 'Communities and Engagement', 'category': 'Service-Learning C&E courses'}, page_content='Group: CHS Common Core Modules\nPillar Name: Communities and Engagement\nCategory: Service-Learning C&E courses\nGen-Coded:  \nModules: GEN2050X (Teach SG), GEN2060X (Reconnect SeniorsSG), GEN2061X (Support Healthy AgeingSG), GEN2062X (Community Activities for Seniors with SG Cares), GEN2070X (Community Link (Comlink) Befrienders)'), Document(id='f487f0f1-64ce-4323-8ac5-b3c154ff1cda', metadata={'group': 'CHS Common Core Modules', 'pillar': 'Communities and Engagement', 'category': 'Project-based Engagement-Learning C&E

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


[Document(id='0d46abcd-7bb5-4549-8365-3d11c54665c9', metadata={'group': 'CHS Integrated Modules', 'pillar': 'Scientific Inquiry II'}, page_content='Group: CHS Integrated Modules\nPillar: Scientific Inquiry II\nModules: HSI20XX'), Document(id='7be994ef-3bd0-4300-92fd-8fcf5449e62c', metadata={'group': 'CHS Common Core Modules', 'pillar': 'Communities and Engagement', 'category': 'Service-Learning C&E courses'}, page_content='Group: CHS Common Core Modules\nPillar Name: Communities and Engagement\nCategory: Service-Learning C&E courses\nGen-Coded:  \nModules: GEN2050X (Teach SG), GEN2060X (Reconnect SeniorsSG), GEN2061X (Support Healthy AgeingSG), GEN2062X (Community Activities for Seniors with SG Cares), GEN2070X (Community Link (Comlink) Befrienders)'), Document(id='f487f0f1-64ce-4323-8ac5-b3c154ff1cda', metadata={'group': 'CHS Common Core Modules', 'pillar': 'Communities and Engagement', 'category': 'Project-based Engagement-Learning C&E courses'}, page_content='Group: CHS Common Core 

...

[Document(id='0d46abcd-7bb5-4549-8365-3d11c54665c9', metadata={'group': 'CHS Integrated Modules', 'pillar': 'Scientific Inquiry II'}, page_content='Group: CHS Integrated Modules\nPillar: Scientific Inquiry II\nModules: HSI20XX'), Document(id='7be994ef-3bd0-4300-92fd-8fcf5449e62c', metadata={'group': 'CHS Common Core Modules', 'pillar': 'Communities and Engagement', 'category': 'Service-Learning C&E courses'}, page_content='Group: CHS Common Core Modules\nPillar Name: Communities and Engagement\nCategory: Service-Learning C&E courses\nGen-Coded:  \nModules: GEN2050X (Teach SG), GEN2060X (Reconnect SeniorsSG), GEN2061X (Support Healthy AgeingSG), GEN2062X (Community Activities for Seniors with SG Cares), GEN2070X (Community Link (Comlink) Befrienders)'), Document(id='f487f0f1-64ce-4323-8ac5-b3c154ff1cda', metadata={'group': 'CHS Common Core Modules', 'pillar': 'Communities and Engagement', 'category': 'Project-based Engagement-Learning C&E courses'}, page_content='Group: CHS Common Core 

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


[Document(id='0d46abcd-7bb5-4549-8365-3d11c54665c9', metadata={'group': 'CHS Integrated Modules', 'pillar': 'Scientific Inquiry II'}, page_content='Group: CHS Integrated Modules\nPillar: Scientific Inquiry II\nModules: HSI20XX'), Document(id='7be994ef-3bd0-4300-92fd-8fcf5449e62c', metadata={'group': 'CHS Common Core Modules', 'pillar': 'Communities and Engagement', 'category': 'Service-Learning C&E courses'}, page_content='Group: CHS Common Core Modules\nPillar Name: Communities and Engagement\nCategory: Service-Learning C&E courses\nGen-Coded:  \nModules: GEN2050X (Teach SG), GEN2060X (Reconnect SeniorsSG), GEN2061X (Support Healthy AgeingSG), GEN2062X (Community Activities for Seniors with SG Cares), GEN2070X (Community Link (Comlink) Befrienders)'), Document(id='f487f0f1-64ce-4323-8ac5-b3c154ff1cda', metadata={'group': 'CHS Common Core Modules', 'pillar': 'Communities and Engagement', 'category': 'Project-based Engagement-Learning C&E courses'}, page_content='Group: CHS Common Core 

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


=== END of stream trace ===
The information you've provided about the **GEN2050X** module is already detailed and includes its title, description, credit value, faculty, department, preclusion rules, and other metadata. Since no specific question or request was included in your message, I’m unable to proceed further. 

Could you clarify how I can assist you? For example:
- Are you looking for **prerequisites** or **timetable details** for this module?
- Do you want to check if it aligns with specific **CHS pillar requirements**?
- Or is there another query related to this module?

Let me know! 😊


In [73]:
reset_chat()

Chat state reset.
